# Lab 3: Supervised Learning - Classification Algorithms

### Aidan Barbieux and Eric Inman

In [2]:
import numpy as np
import pandas as pd
import math

## C4.5 Algorithm

In [ ]:
# entropy of a series of data
def entropy(classcol):
    vals = classcol.value_counts()
    size = classcol.count()
    entropy=0
    for v in vals:
        entropy -= (v/size) * math.log(v/size,2)
    return entropy

# entropy of an attribute in a dataset, over each value of the attribute
def entropyAttr(data, attr):
    vals = data.pivot(columns=attr,values=data.columns[-1])
    entropyTot = 0
    for c in vals.columns:
        entropyTot += (vals[c].count()/len(data)) * entropy(vals[c])
    return entropyTot

In [ ]:
def selectSplittingAttr(attrs, data, threshold):
    p0 = entropy(data.iloc[:,-1])
    gain={}
    for a in attrs:
        gain[a] = p0 - entropyAttr(data, a) # info gain
    
    bestAttr=max(gain,key=gain.get)
    if gain[bestAttr] > threshold:
        return bestAttr
    else:
        return None

### C4.5

In [15]:
# class must be in last column
def c45(data, attrs, tree, thresh):
    # base case 1
    classes = data.iloc[:,-1]
    firstclass = None
    allsame=True
    for c in classes:
        if c == None:
            firstclass = c
        elif c != firstclass:
            allsame=False
            break
    if allsame:
        #create leaf node for perfect purity
        tree["leaf"] = {
            "decision": firstclass,
            "p": 1.0
        }
    
    # base case 2
    if len(attrs) == 0:
        # create leaf node with most frequent class
        tree["leaf"] = {
            "decision": classes.mode(),
            "p": classes.value_counts()[classes.mode()]/len(classes)
        }
    
    # select splitting attr
    asplit = selectSplittingAttr(attrs, data, thresh)
    if asplit = None:
        tree["leaf"] = {
            "decision": classes.mode(),
            "p": classes.value_counts()[classes.mode()]/len(classes)
        }
    else:
        newNode = {}
        # populate node w c45 recursively, add newNode to tree
        
#     # select splitting attr

df = readArrange('adult-stretch.csv')

tree={"dataset": 'adult-stretch.csv'}
c45(df, df.columns[:-1], tree, 0.0)        
# def addNode(tree,tf):
#     if tf:
#         tree["node"] = {"var": 'blah', "edges": []}
#     else:
#         tree["leaf"] = {"decision": 'yes', "p": 0.7}
        
# t = {}
# t["dataset"] = ".csv"
# t["node"] = {"var": 'blah', "edges": []}
# for i in range(10):
#     tmp = {}
#     addNode(tmp, True if i%2 == 0 else False)
#     tmp["value"]=str(i)
#     t["node"]["edges"].append({"edge":tmp})
    
# t
 

SyntaxError: invalid syntax (<ipython-input-15-2c0b36249972>, line 27)

In [6]:
def readArrange(filename):
    df = pd.read_csv(filename)
    aclass = df.iloc[1,0]
    df = df.drop([0,1], axis=0)
    df = df[[c for c in df if c not in [aclass]] + [aclass]]
    return df

## Data